In [70]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import pandas as pd

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

### IMPORTS ###
import math;
import sys;
import itertools;
import string
import random
import zlib
import struct
import matplotlib.pyplot as plt
import struct
from array import array
from os.path import join
import sklearn
import sklearn.neighbors
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

### SETTING DEFAULTS ###
sys.setrecursionlimit(20000)

/kaggle/input/banana/banana_quality.csv


# 6.6 1a

a. Show empirically that the information limit of 2 prediction bits per parameter also holds for nearest neighbors.

TODO: Implement KNN and show that if the dimension is high enough, we can get 100% accuracy on the test set by using half the data points in the training. Overfitting happens very easily in higher dimensions.
Process: Create a set of 100 value. Remove half. Train KNN on half with a very high d. The accuracy of the half that we removed will still be 100%. If the half we remove is more than half, then our accuracy will not be 100%.

In [129]:
#### SETUP CODE ####
def num_functions_iterative(n,d):
    return 2*sum([math.comb(n-1, l) for l in range (0, d)])

def num_functions_recursive(n,d, cache = dict()):
    if (n,d) in cache:
        return cache[(n,d)]
    if n == 1 or d == 1:
        ### BASE CASE HAS 2 CLASSIFICATIONS
        cache[(n,d)] = 2;
        return 3
    elif n == 0 or d == 0:
        cache[(n,d)] = 0;
        return 0
    res = num_functions_recursive(n - 1, d, cache) + num_functions_recursive(n - 1, d - 1, cache);
    cache[(n,d)] = res
    return res
        
def conduct_experiment(dimensions, num_classes):
    np.random.seed(10)
    
    for d in dimensions:
        n_full = 2 ** d
        row_limit = 2 ** 8
        X = np.random.randint(0, 2, size=(n_full, d))
        total_rows = min(num_classes ** n_full, row_limit)
        Y = np.random.randint(0, num_classes, size=(total_rows, n_full))
        average_number_of_points_memorized = 0
        
        for Y_row in Y:
            function = np.concatenate((X, Y_row.reshape(-1, 1)), axis=1)
            for n_pts_to_train_on in range(1, n_full):
                indices = np.random.choice(range(n_full), n_pts_to_train_on, replace=False)
                X_train = function[indices, :-1]
                Y_train = function[indices, -1]
                X_test = np.delete(function, indices, axis=0)[:, :-1]
                Y_test = np.delete(function, indices, axis=0)[:, -1]
                
                knn = KNeighborsClassifier(n_neighbors=1)
                knn.fit(X_train, Y_train)
                Y_pred = knn.predict(X_test)
                accuracy = accuracy_score(Y_test, Y_pred)
                
                if accuracy == 1.0:
                    average_number_of_points_memorized += n_pts_to_train_on
        
        average_number_of_points_memorized /= total_rows
        print(f"d={d}: n_full={n_full}, Avg. req. points for memorization={average_number_of_points_memorized}, n_full/n_avg={n_full/average_number_of_points_memorized}")

In [130]:
conduct_experiment([1, 2, 3, 4], 2)

d=1: n_full=2, Avg. req. points for memorization=0.25, n_full/n_avg=8.0
d=2: n_full=4, Avg. req. points for memorization=1.3125, n_full/n_avg=3.0476190476190474
d=3: n_full=8, Avg. req. points for memorization=5.8984375, n_full/n_avg=1.3562913907284768
d=4: n_full=16, Avg. req. points for memorization=13.05859375, n_full/n_avg=1.2252467843254562


# 6.6 1b

b. Extend your experiments to multi-class classification.

In [114]:
conduct_experiment([2,4,8],3)

d=2: n_full=4 Avg. req. points for memorization=1.8024691358024691, n_full/n_avg=2.219178082191781
d=4: n_full=16 Avg. req. points for memorization=7.171875, n_full/n_avg=2.2309368191721135


KeyboardInterrupt: 

# 6.6 2a

Implement a program that automatically creates a set of if-then clauses from the training table of a binary dataset of your choice. Implement different strategies to minimize the number of if-then clauses. Document your strategies, the number of resulting conditional clauses, and the accuracy achieved.

In [74]:
# Import rulekit package that allows us to get the rules used by the classifier
!pip install rulekit
!python -m rulekit download_jar

Old RuleKit jar file/files (['rulekit-1.7.5-all.jar']) detected,do you want to remove it/them? Type "yes" or "no"
^C
Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/opt/conda/lib/python3.10/runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "/opt/conda/lib/python3.10/site-packages/rulekit/__main__.py", line 60, in <module>
    _main()
  File "/opt/conda/lib/python3.10/site-packages/rulekit/__main__.py", line 51, in _main
    _download_rulekit_jar()
  File "/opt/conda/lib/python3.10/site-packages/rulekit/__main__.py", line 28, in _download_rulekit_jar
    user_input: str = input(
KeyboardInterrupt


In [96]:
# SETUP -- https://adaa-polsl.github.io/RuleKit-python/
from rulekit.classification import RuleClassifier
from rulekit.params import Measures
import sklearn.tree as scikit
from sklearn.datasets import load_breast_cancer
import math
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn import metrics
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split


def get_prediction_metrics(measure: str, y_pred, y_true, classification_metrics: dict) -> tuple[pd.DataFrame, np.ndarray]:
    confusion_matrix = metrics.confusion_matrix(y_true, y_pred)
    tn, fp, fn, tp = confusion_matrix.ravel()
    sensitivity = tp  / (tp + fn)
    specificity = tn  / (tn + fp)
    npv = tn  / (tn + fn)
    ppv = tp  / (tp + fp)

    dictionary = {
        'Measure': measure,
        'Accuracy': metrics.accuracy_score(y_true, y_pred),
        'MAE': metrics.mean_absolute_error(y_true, y_pred),
        'Kappa': metrics.cohen_kappa_score(y_true, y_pred),
        'Balanced accuracy': metrics.balanced_accuracy_score(y_true, y_pred),
        'Logistic loss': metrics.log_loss(y_true, y_pred),
        'Precision': metrics.log_loss(y_true, y_pred),
        'Sensitivity': sensitivity,
        'Specificity': specificity,
        'NPV': npv,
        'PPV': ppv,
        'psep': ppv + npv - 1,
        'Fall-out': fp  / (fp + tn),
        "Youden's J statistic": sensitivity + specificity - 1,
        'Lift': (tp / (tp + fp)) / ((tp + fn) / (tp + tn + fp + fn)),
        'F-measure': 2 * tp / (2 * tp + fp + fn),
        'Fowlkes-Mallows index': metrics.fowlkes_mallows_score(y_true, y_pred),
        'False positive': fp,
        'False negative': fn,
        'True positive': tp,
        'True negative': tn,
        'Rules per example': classification_metrics['rules_per_example'],
        'Voting conflicts': classification_metrics['voting_conflicts'],
        'Negative voting conflicts': classification_metrics['negative_voting_conflicts'],
        'Geometric mean': math.sqrt(specificity * sensitivity),
        'Geometric mean': math.sqrt(specificity * sensitivity),
    }
    return pd.DataFrame.from_records([dictionary], index='Measure'), confusion_matrix

def get_ruleset_stats(measure: str, model) -> pd.DataFrame:
    return pd.DataFrame.from_records([{'Measure': measure, **model.stats.__dict__}], index='Measure')

def evaluator(X,y, onlyReturnAccuracy=False):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0);
    # C2
    clf = RuleClassifier(
        induction_measure=Measures.C2,
        pruning_measure=Measures.C2,
        voting_measure=Measures.C2,
    )
    clf.fit(X_train, y_train)
    c2_ruleset = clf.model
    prediction, classification_metrics = clf.predict(X_test, return_metrics=True)

    prediction_metric, c2_confusion_matrix = get_prediction_metrics('C2', prediction, y_test, classification_metrics)
    model_stats = get_ruleset_stats('C2', clf.model)

    # RSS
    clf = RuleClassifier(
        induction_measure=Measures.RSS,
        pruning_measure=Measures.RSS,
        voting_measure=Measures.RSS,
    )
    clf.fit(X_train, y_train)
    rss_ruleset = clf.model
    prediction, classification_metrics = clf.predict(X_test, return_metrics=True)
    tmp, rss_confusion_matrix = get_prediction_metrics('RSS', prediction, y_test, classification_metrics)
    prediction_metric = pd.concat([prediction_metric, tmp])
    model_stats = pd.concat([model_stats, get_ruleset_stats('RSS', clf.model)])

    if onlyReturnAccuracy:
        display(prediction_metric[["Accuracy"]])
    else:
        print("c2 Rules: ")
        for rule in c2_ruleset.rules:
            print(rule)
        print("\nRSS Rules: ")
        for rule in rss_ruleset.rules:
            print(rule)
        display(model_stats)
        display(prediction_metric)
    

**Comparing CE Loss, Residual Sum of Squares, and Correlation for Cancer Data**
This data is relatively small (569 rows)

In [91]:
# Data description
cancer_data = load_breast_cancer()
X = pd.DataFrame(data=cancer_data.data,columns=cancer_data.feature_names)
X.describe()

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst radius,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension
count,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,...,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000
mean,14.127292,19.289649,91.969033,654.889104,0.096360,0.104341,0.088799,0.048919,0.181162,0.062798,...,16.269190,25.677223,107.261213,880.583128,0.132369,0.254265,0.272188,0.114606,0.290076,0.083946
std,3.524049,4.301036,24.298981,351.914129,0.014064,0.052813,0.079720,0.038803,0.027414,0.007060,...,4.833242,6.146258,33.602542,569.356993,0.022832,0.157336,0.208624,0.065732,0.061867,0.018061
min,6.981000,9.710000,43.790000,143.500000,0.052630,0.019380,0.000000,0.000000,0.106000,0.049960,...,7.930000,12.020000,50.410000,185.200000,0.071170,0.027290,0.000000,0.000000,0.156500,0.055040
25%,11.700000,16.170000,75.170000,420.300000,0.086370,0.064920,0.029560,0.020310,0.161900,0.057700,...,13.010000,21.080000,84.110000,515.300000,0.116600,0.147200,0.114500,0.064930,0.250400,0.071460
50%,13.370000,18.840000,86.240000,551.100000,0.095870,0.092630,0.061540,0.033500,0.179200,0.061540,...,14.970000,25.410000,97.660000,686.500000,0.131300,0.211900,0.226700,0.099930,0.282200,0.080040
75%,15.780000,21.800000,104.100000,782.700000,0.105300,0.130400,0.130700,0.074000,0.195700,0.066120,...,18.790000,29.720000,125.400000,1084.000000,0.146000,0.339100,0.382900,0.161400,0.317900,0.092080
max,28.110000,39.280000,188.500000,2501.000000,0.163400,0.345400,0.426800,0.201200,0.304000,0.097440,...,36.040000,49.540000,251.200000,4254.000000,0.222600,1.058000,1.252000,0.291000,0.663800,0.207500


In [92]:
cancer_data = load_breast_cancer()
X = pd.DataFrame(data=cancer_data.data,columns=cancer_data.feature_names) 
y = cancer_data.target
evaluator(X,y)

c2 Rules: 
IF worst perimeter = (-inf, 101.65) AND worst concave points = (-inf, 0.18) AND area error = (-inf, 48.98) AND worst smoothness = (-inf, 0.19) THEN label = {1}
IF worst concave points = (-inf, 0.11) AND mean radius = (-inf, 14.98) AND area error = (-inf, 48.98) THEN label = {1}
IF radius error = (-inf, 1.05) AND worst concave points = (-inf, 0.11) AND mean perimeter = (-inf, 106.85) THEN label = {1}
IF area error = (-inf, 46.32) AND mean concave points = (-inf, 0.045) AND worst radius = (-inf, 16.83) AND smoothness error = <0.0033, inf) THEN label = {1}
IF area error = (-inf, 42.19) AND worst smoothness = (-inf, 0.18) AND worst area = (-inf, 957.45) AND mean concave points = (-inf, 0.049) THEN label = {1}
IF worst concave points = (-inf, 0.16) AND worst smoothness = (-inf, 0.19) AND worst area = (-inf, 768.10) AND perimeter error = (-inf, 4.11) THEN label = {1}
IF worst perimeter = <117.45, inf) AND mean symmetry = <0.13, inf) THEN label = {0}
IF worst perimeter = <114.45, i

,time_total_s,time_growing_s,time_pruning_s,rules_count,conditions_per_rule,induced_conditions_per_rule,avg_rule_coverage,avg_rule_precision,avg_rule_quality,pvalue,FDR_pvalue,FWER_pvalue,fraction_significant,fraction_FDR_significant,fraction_FWER_significant
Measure,,,,,,,,,,,,,,,
C2,0.139516,0.131531,0.003409,13,4.153846,5.846154,0.409129,0.988899,0.889438,1.136452e-58,1.136452e-58,1.136452e-58,1.0,1.0,1.0
RSS,0.172540,0.165015,0.004099,7,5.000000,11.428571,0.491994,0.966164,0.879027,2.198988e-86,2.298724e-86,2.769373e-86,1.0,1.0,1.0


,Accuracy,MAE,Kappa,Balanced accuracy,Logistic loss,Precision,Sensitivity,Specificity,NPV,PPV,...,F-measure,Fowlkes-Mallows index,False positive,False negative,True positive,True negative,Rules per example,Voting conflicts,Negative voting conflicts,Geometric mean
Measure,,,,,,,,,,,,,,,,,,,,,
C2,0.964912,0.035088,0.927596,0.963798,1.264690,1.264690,0.970149,0.957447,0.957447,0.970149,...,0.970149,0.933171,2,2,65,45,5.350877,14.0,2.0,0.963777
RSS,0.938596,0.061404,0.873694,0.938234,2.213207,2.213207,0.940299,0.936170,0.916667,0.954545,...,0.947368,0.885914,3,4,63,44,3.350877,12.0,8.0,0.938232


**Conclusions**
C2 and RSS metrics produce similar accuracies on the test set. C2 produces an accuracy of 96% wby generating 13 rules.
RSS produces an accuracy of 93% by generating 7 rules. We can also see that the average rule coverage of the RSS rule is much higher than that of C2 which shows higher generalization. Similarly, the average rule precision of a C2 rule is higher than that of an RSS rule which also makes sesne.

# 6.6 2b

Use the algorithms developed in (a) on different datasets. Again, observe how your choices make a difference.

**Comparing CE Loss, Residual Sum of Squares, and Correlation for Banana Quality Data** This dataset is more than 10 times larger than the cancer dataset (8000 rows)

In [83]:
data_set = pd.read_csv("/kaggle/input/banana/banana_quality.csv")
data_set.describe()

,Size,Weight,Sweetness,Softness,HarvestTime,Ripeness,Acidity
count,8000.000000,8000.000000,8000.000000,8000.000000,8000.000000,8000.000000,8000.000000
mean,-0.747802,-0.761019,-0.770224,-0.014441,-0.751288,0.781098,0.008725
std,2.136023,2.015934,1.948455,2.065216,1.996661,2.114289,2.293467
min,-7.998074,-8.283002,-6.434022,-6.959320,-7.570008,-7.423155,-8.226977
25%,-2.277651,-2.223574,-2.107329,-1.590458,-2.120659,-0.574226,-1.629450
50%,-0.897514,-0.868659,-1.020673,0.202644,-0.934192,0.964952,0.098735
75%,0.654216,0.775491,0.311048,1.547120,0.507326,2.261650,1.682063
max,7.970800,5.679692,7.539374,8.241555,6.293280,7.249034,7.411633


In [89]:
data_set = pd.read_csv("/kaggle/input/banana/banana_quality.csv")
data_set['Quality'] = data_set['Quality'].apply(lambda x: 1 if x == 'Good' else 0)
X = data_set.iloc[:, 0:-1]
y = data_set.iloc[:, -1]
evaluator(X,y)

c2 Rules: 
IF HarvestTime = <-3.73, inf) AND Size = <-4.65, inf) AND Sweetness = <1.91, inf) AND Softness = (-inf, 1.20) AND Weight = <-0.83, inf) THEN Quality = {1}
IF Size = <-4.65, inf) AND Sweetness = <1.91, inf) AND Softness = (-inf, 1.20) AND Weight = <-0.83, inf) THEN Quality = {1}
IF HarvestTime = <-3.73, inf) AND Size = <-4.65, inf) AND Sweetness = <1.91, inf) AND Softness = <-3.33, 1.20) AND Weight = <-4.10, inf) THEN Quality = {1}
IF HarvestTime = <-3.74, inf) AND Sweetness = <1.91, inf) AND Softness = (-inf, 1.20) AND Weight = <-0.45, inf) THEN Quality = {1}
IF HarvestTime = <-3.73, inf) AND Size = <-4.65, inf) AND Sweetness = <1.91, inf) AND Softness = (-inf, 1.25) AND Acidity = <-1.05, inf) AND Weight = <-4.10, inf) THEN Quality = {1}
IF HarvestTime = <-3.70, 1.63) AND Size = <-3.63, inf) AND Sweetness = <1.91, inf) AND Softness = <-3.33, inf) AND Weight = <-2.58, inf) THEN Quality = {1}
IF HarvestTime = <-3.70, inf) AND Size = <-3.23, inf) AND Sweetness = <1.91, inf) AND

,time_total_s,time_growing_s,time_pruning_s,rules_count,conditions_per_rule,induced_conditions_per_rule,avg_rule_coverage,avg_rule_precision,avg_rule_quality,pvalue,FDR_pvalue,FWER_pvalue,fraction_significant,fraction_FDR_significant,fraction_FWER_significant
Measure,,,,,,,,,,,,,,,
C2,13.314546,11.793684,1.468723,208,4.485577,15.697115,0.094123,0.981124,0.566565,6.340617e-05,6.358105e-05,9.943036e-05,1.0,1.0,1.0
RSS,45.597431,31.136052,14.442076,27,4.000000,219.000000,0.405098,0.804238,0.468175,4.654947e-191,4.654947e-191,4.654947e-191,1.0,1.0,1.0


,Accuracy,MAE,Kappa,Balanced accuracy,Logistic loss,Precision,Sensitivity,Specificity,NPV,PPV,...,F-measure,Fowlkes-Mallows index,False positive,False negative,True positive,True negative,Rules per example,Voting conflicts,Negative voting conflicts,Geometric mean
Measure,,,,,,,,,,,,,,,,,,,,,
C2,0.9475,0.0525,0.895006,0.947553,1.892292,1.892292,0.954660,0.940447,0.954660,0.940447,...,0.947500,0.900394,48,36,758,758,19.027500,500.0,210.0,0.947527
RSS,0.8125,0.1875,0.624063,0.811271,6.758185,6.758185,0.647355,0.975186,0.737336,0.962547,...,0.774096,0.711917,20,280,514,786,11.083125,1180.0,860.0,0.794539


**Conclusions**
C2 and RSS metrics produce very different accuracies on the train and test set. C2 produces an accuracy of 94% wby generating 208 rules.
RSS produces an accuracy of 81% by generating 27 rules. We can also see that the average rule coverage of the RSS rule is much higher than that of C2 which shows higher generalization.

# 6.6 2c

Finally, use the programs developed in (a) on a completely random dataset, generated artificially. Vary your strategies but also the number of input columns as well as the number of instances. How many if-then clauses do you need?



**Experiment 1 - Using different measures for pruning and indiction - comparing C2 and RSS**

In [104]:
from IPython.display import display
import sklearn.tree as scikit
from sklearn.datasets import load_iris
import math
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn import metrics
import pandas as pd
import numpy as np

dimensions = [2, 4, 8, 16]

for n in [100, 500, 1000]:
    for d in [2, 4, 8, 16]:
        print(f"d={d}, n={n}")
        random_dataset = np.random.rand(n, d)
        random_labels = np.random.randint(0, 2, size=n)
        evaluator(random_dataset,random_labels, onlyReturnAccuracy = True)

d=2, n=100


,Accuracy
Measure,
C2,0.25
RSS,0.35


d=4, n=100


,Accuracy
Measure,
C2,0.65
RSS,0.85


d=8, n=100


...

,Accuracy
Measure,
C2,0.515
RSS,0.485


d=8, n=1000


,Accuracy
Measure,
C2,0.435
RSS,0.430


d=16, n=1000


,Accuracy
Measure,
C2,0.45
RSS,0.54


**Conclusions**
For a random dataset, produce similar average accuracies of about 50% even when the number of samples are varied as well as the dimensions are varied.

# 6.6 3a

Create a long random string using a Python program, and use a lossless compression algorithm of your choice to compress the string. Note the compression ratio.



In [19]:
##### Question 6 3a ######
random.seed(40)
string_length = int(1);
random_string = ''.join(random.choices(string.ascii_uppercase + string.digits, k=string_length))
compressed = zlib.compress(random_string.encode())
uncompressed = zlib.decompress(compressed)
compression_ratio = sys.getsizeof(random_string) / sys.getsizeof(compressed)
print(compression_ratio)

1.1904761904761905


# 6.6 3b

What is the expected compression ratio? Explain why.



The expected compression ratio of a truly random string is 1 because the string cannot be compressed. The output above is greater than 1 because the string that I am producing via random.choices isn't a truly random string but rather a pseudo random string that can be compressed.